# Testy statystyczne. Część III

In [1]:
import pandas as pd
from pandas import DataFrame, Series
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg
from pprint import pprint

## Prosta regresja liniowa

### Przykład

Wróćmy do naszego badania dotyczącego związku wieku i skłonności do akceptacji zdań przypisujacych wiedzę. Załóżmy, że interesuje nas nie tylko to, czy te dwie własności są ze sobą skorelowane, ale również w jaki sposób. Aby się tego dowiedzieć przeprowadzimy regresję liniową. Za pomocą modułu `statsmodels.formula.api` mozemy wykonać ją w sposób niemal identyczny jak w przypadku R. Za pomocą funkcji `ols` konstruujemy model, `fit` dopasowuje go do danych a `summary` wyświetla informacje o naszej regresji.

Formuła dla każdego modelu ma postać `zmienna_zależna ~ predyktor1 + predyktor2 + ...`

In [2]:
data = pd.read_csv('risk.csv')

In [3]:
smf.ols('rating ~ age', data = data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     14.52
Date:                Mon, 01 Apr 2019   Prob (F-statistic):           0.000242
Time:                        00:52:13   Log-Likelihood:                -197.81
No. Observations:                 100   AIC:                             399.6
Df Residuals:                      98   BIC:                             404.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.7460      0.650      4.224      0.000       1.456       4.036
age            0.0905      0.024      3.811      0.000       0.043       0.138
==============================================================================
Omnibus:                        0.405   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.817   Jarque-Bera (JB):                0.142
Skew:                          -0.076   Prob(JB):                        0.931
Kurtosis:                       3.106   Cond. No.                         101.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Widzimy, że współczynnik regresji wynosi $0.09$. W przypadku naszego badania jest to informacja niekorzystna dla badaczy - okazuje się, że zwiazek istnieje, ale jest odwrotny niż postulowany! Wiek zwiększa skłonność do przypisywania wiedzy w sytuacjach ryzyka epistemicznego!

Taką samą analizę możemy wykonać używając funkcji `linear_regression` z modułu `pingouin.regression`: 

In [4]:
pg.regression.linear_regression(y = data['rating'], # zmienna zależna 
                                X = data['age']) # predyktor

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,2.746016,0.650098,4.224003,0.000054,0.129047,0.12016,1.455917,4.036114
1,age,0.090543,0.023761,3.810569,0.000242,0.129047,0.12016,0.043390,0.137695


## Prosta analiza wariancji (jednoczynnikowa)

### Przykład

Tym razem wróćmy do naszego badania dotyczącego interpretacji zdań w rodzaju "Chłopcy palą papierosy". Załóżmy, że zamiast testu Kruskala-Wallisa chcielibyśmy wykonać zwykłą jednoczynnikową analizę wariancji (ponieważ doszliśmy do wniosku, że wszystkie jej założenia są jednak spełnione). Mozemy zrobić to za pomocą odpowiedniej funkcji z pakietu `statsmodels`:

In [5]:
data = pd.read_csv('elektrycznie_gitary.csv')
model = smf.ols('sent_judg ~ condition', data = data).fit()
sm.stats.anova_lm(model, type=2) # zwykle będziemy chcieli ANOVA typu 2

,df,sum_sq,mean_sq,F,PR(>F)
condition,2.0,346.016667,173.008333,132.603832,8.401532e-31
Residual,117.0,152.650000,1.304701,NaN,NaN


Przy użyciu ANOVA również okazało się (nic dziwnego) że typ zdania ma istotny wpływ na ocenę prawdziwości tego zdania w testowanym przez nas kontekście.

Analizę tę wykonać możemy również za pomocą funkcji `anova` z pakietu `pingouin`. Wróci nam ona dodatkowo informacje o wielkości efektu (który jest tutaj b. duży).

In [6]:
pg.anova(data = data, between = 'condition', dv = 'sent_judg')

,Source,ddof1,ddof2,F,p-unc,np2
0,condition,2,117,132.604,8.401532e-31,0.694


## Wieloraka regresja liniowa

### Przykład

Nasi badacze analizujący przypisania wiedzy nie poddali się i postanowili przeprowadzić *follow-up* do poprzedniego eksperymentu, w którym dodatkowo oprócz informacji o wieku zbierali:
+ informacje o inteligencji badanego
+ informacje o płci badanego

Ich hipoteza głosiła, że inteligencja generalnie wpływać będzie ujemnie na przypisanie wiedzy, podczas gdy w przypadku kobiet będzie wpływać dodatnio. Dane z eksperymentu znajdują się w pliku `risk-extended.csv`.

W skonstruowanym niżej modelu testujemy wpływ: 
+ wieku
+ interakcji czynników - płci i IQ

In [7]:
data = pd.read_csv('risk_extended.csv')
data.head()

,Unnamed: 0,age,rating,participant,sex,iq
0,0,36.0,5.0,1,M,107.192039
1,1,34.0,6.0,2,K,81.352211
2,2,30.0,8.0,3,M,79.571084
3,3,34.0,6.0,4,K,80.494001
4,4,22.0,7.0,5,K,86.378892


In [8]:
smf.ols('rating ~ age + sex:iq', data = data).fit().summary() # predyktor1:predyktor2 oznacza interakcję między nimi

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     6.142
Date:                Mon, 01 Apr 2019   Prob (F-statistic):           0.000726
Time:                        00:52:25   Log-Likelihood:                -186.71
No. Observations:                 100   AIC:                             381.4
Df Residuals:                      96   BIC:                             391.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.0810      1.093      4.650      0.000       2.912       7.250
age            0.0699      0.019      3.733      0.000       0.033       0.107
sex[K]:iq     -0.0196      0.010     -1.941      0.055      -0.040       0.000
sex[M]:iq     -0.0117      0.010     -1.188      0.238      -0.031       0.008
==============================================================================
Omnibus:                        3.875   Durbin-Watson:                   2.144
Prob(Omnibus):                  0.144   Jarque-Bera (JB):                3.541
Skew:                          -0.460   Prob(JB):                        0.170
Kurtosis:                       3.046   Cond. No.                         526.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Niestety znów pudło! Hipoteza badaczy okazała się być częściowo prawdziwa. Zaobserwować możemy wpływ wieku na odpowiedzi, ale IQ x płeć nie wpływa na przypisywanie wiedzy. 

Podobną (ale nie taką samą!) analizę możemy wykonać za pomocą `linear_regression` z pakietu `pingoin`. Tym razem warto zwrócić uwage, że nie testuje ona interakcji między czynnikami.

In [9]:
# Tworzymy sobie zmienna liczbową do zakodowania płci
data['sex_num'] = data['sex'].map({'M' : 1, 'K' : 0})
pg.linear_regression(y=data['rating'], X = data[['age', 'sex_num', 'iq']])

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,4.766232,1.107023,4.305450,0.000040,0.154141,0.127708,2.568809,6.963655
1,age,0.069029,0.018783,3.675006,0.000392,0.154141,0.127708,0.031744,0.106314
2,sex_num,0.751626,0.331696,2.266009,0.025697,0.154141,0.127708,0.093215,1.410037
3,iq,-0.015967,0.009885,-1.615187,0.109552,0.154141,0.127708,-0.035589,0.003656


Poniżej to samo za pomocą `statsmodels`. Co kto lubi :)

In [10]:
smf.ols('rating ~ age + sex + iq', data = data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rating   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     5.831
Date:                Mon, 01 Apr 2019   Prob (F-statistic):            0.00106
Time:                        00:54:57   Log-Likelihood:                -187.12
No. Observations:                 100   AIC:                             382.2
Df Residuals:                      96   BIC:                             392.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.7662      1.107      4.305      0.000       2.569       6.964
sex[T.M]       0.7516      0.332      2.266      0.026       0.093       1.410
age            0.0690      0.019      3.675      0.000       0.032       0.106
iq            -0.0160      0.010     -1.615      0.110      -0.036       0.004
==============================================================================
Omnibus:                        3.857   Durbin-Watson:                   2.168
Prob(Omnibus):                  0.145   Jarque-Bera (JB):                3.547
Skew:                          -0.461   Prob(JB):                        0.170
Kurtosis:                       3.029   Cond. No.                         721.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Wieloczynnikowa analiza wariancji

### Przykład

Tym razem znów wróćmy do chłopców palących papierosy. Badacze zastanawiali się nad tym, czy zaobserwowany (dość oczywisty) wzorzec odpowiedzi można zaobserwować także w innym niż polski języku. Przeprowadzili więc badanie na 120 rodzimych użytkownikach polskiego oraz 120 rodzimych użytkownikach angielskiego. Wyniki znajdują się w pliku `elektryczne_gitary_extended.xlsx`. Przeprowadźmy wieloczynnikową analizę wariancji, aby poznać odpowiedź na ich pytanie badawcze.

In [11]:
data = pd.read_excel('elektryczne_gitary_extended.xlsx')
data.head()

,Unnamed: 0,participant,condition,sent_judg,lang
0,0,1,universal,3,pl
1,1,2,existential,7,pl
2,2,3,no-quantifier,5,pl
3,3,4,universal,3,pl
4,4,5,existential,6,pl


In [12]:
model = smf.ols('sent_judg ~ condition * lang', data = data).fit() # predyktor1 * predyktor2 to efekt obu + interakcji
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
condition,2.0,689.275000,344.637500,210.328421,5.304580e-53
lang,1.0,11.704167,11.704167,7.142922,8.055514e-03
condition:lang,2.0,0.558333,0.279167,0.170372,8.434553e-01
Residual,234.0,383.425000,1.638568,NaN,NaN


Widzimy, że język faktycznie jest istotnym czynnikiem ($p<0.05$) jednak nie odnaleziono żadnej statystycznie istotnej interakcji między językiem i warunkiem eksperymentalnym. Okazuje się, że użytkownicy angielskiego po prostu silniej akceptują zdanie o chłopcach w przedstawionym kontekście, niezależnie od jego budowy. Jak uprzejmie z ich strony! 

## Analiza wariancji dla czynników wewnątrzgrupowych

### Przykład

Przeprowadziliśmy eksperyment mający na celu przetestowanie efektu Stroopa. Nasi badani mieli powiedzieć jakiego koloru jest słowo. W badaniu były dwa rodzaje prób - takie, w których kolor był zgodny z treścią słowa (np. "czerwony" napisane czerwonym kolorem) lub takie, w których był niezgodny (np. "czerwony" napisane niebieskim kolorem). Chcemy porównać czasy reakcji w tych dwóch warunkach. Wyniki badania znajdują się w pliku `bi-stroop.csv`. Do porównania użyjemy analizy wariancji dla czynników wewnątrzgrupowych. 



In [13]:
data = pd.read_csv('bi-stroop.csv')
data.head()

,Unnamed: 0,participant,condition,rt,lang
0,0,1,congruent,445.882646,pol
1,1,1,congruent,265.583884,pol
2,2,1,incongruent,392.007832,pol
3,3,1,congruent,349.962448,pol
4,4,1,incongruent,188.844854,pol


In [14]:
data.groupby(['condition'])['rt'].describe() # zobaczmy statystyki deskryptywne dotyczące czasów reakcji

,count,mean,std,min,25%,50%,75%,max
condition,,,,,,,,
congruent,2000.0,373.058196,160.882364,89.573234,264.171685,337.557604,444.531468,1259.978454
incongruent,2000.0,434.167958,174.879856,75.032203,320.442374,402.525503,514.128443,2285.353847


In [15]:
from statsmodels.stats.anova import AnovaRM
AnovaRM(data = data, 
        subject='participant', # nazwa kolumny z identyfikatorem badanego
        within=['condition'], # lista z nazwami kolumn zmiennej wewnątrzgrupowych
        depvar='rt', # zmienna zależna
       aggregate_func='mean' # agregujemy do średniej dla każdego badanego
       ).fit().summary()

,Num DF,Den DF,F Value,Pr > F
condition,1.0000,39.0000,110.7458,0.0000


To samo (+ wielkośc efektu) możemy uzyskać za pomocą funkcji `rm_anova` z pakietu `pingouin`.

In [16]:
pg.rm_anova(data = data,
           subject='participant',
           dv = 'rt',
           within='condition')

,Source,ddof1,ddof2,F,p-unc,np2,eps
0,condition,1,39,110.746,5.913191e-13,0.74,1.0


## Analiza wariancji dla schematu mieszanego

### Przykład

Jak (być może) dało się zauważyć, tak naprawdę powyższy eksperyment przeprowadzony był w dwóch wersjach - polskojęzycznej i angielskojęzycznej na populacji rodzimych użytkowników polskiego. Celem eksperymentu było sprawdzenie jak efekt Stroopa zachowuje się w drugim języku. W takim wypadku mamy jeden warunek wewnątrzgrupowy (condition: congruent vs incongruent) i jeden międzygrupowy (lang: pol vs eng). Z tego względu musimy zastosować analizę wariancji dla schematu mieszanego. Zrobimy to za pomocą funkcji `mixed_anova` z pakietu `pingouin`.

In [17]:
data = pd.read_csv('bi-stroop.csv')

In [18]:
data.groupby(['lang', 'condition'])['rt'].describe()

count        mean         std         min         25%  \
lang condition                                                             
eng  congruent    1000.0  391.833963  157.494121  118.681871  284.454624   
     incongruent  1000.0  445.489736  173.994919  101.260486  326.380860   
pol  congruent    1000.0  354.282428  162.117292   89.573234  242.964495   
     incongruent  1000.0  422.846181  175.116211   75.032203  313.002940   

                         50%         75%          max  
lang condition                                         
eng  congruent    349.562281  461.664580  1259.978454  
     incongruent  410.219200  531.946769  1547.244065  
pol  congruent    321.565445  433.029511  1259.406386  
     incongruent  393.809734  495.973376  2285.353847

In [19]:
pg.mixed_anova(data=data, within='condition', between='lang', dv = 'rt', subject='participant')

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,lang,18117.244,1,38,18117.244,37.679,3.678075e-07,0.498,-
1,condition,74688.063,1,38,74688.063,112.666,6.342989e-13,0.748,1
2,Interaction,1111.239,1,38,1111.239,1.676,2.032297e-01,0.042,-


Okazuje się zarówno język jak i typ próby jest statystycznie istotny. Interakcja między tymi czynnikami nie jest - badani polskojęzyczni odpowiadają szybciej niż anglojęzyczni, dodatkowo w przypadku prób "incongruent" czas reakcji jest wyższy.